In [ ]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

def polynomial_regression(degree, X, y, folds, test_size=0.25, random_state=None):
    # Define number of folds for cross-validation
    kf = KFold(folds)

    # Initialize lists to store results for variance, bias2s, total_error, models, r2, and mse
    variance = []
    bias2 = []
    total_error = []
    models = []
    r2 = []
    mse = []

    # Set the polynomial degree of the model
    poly_features = PolynomialFeatures(degree)
    X_poly = poly_features.fit_transform(X)

    # Perform cross-validation
    for train_index, test_index in kf.split(X_poly):
        # Split data into training and testing sets for this fold
        X_train, X_test = X_poly[train_index], X_poly[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Fit polynomial regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Calculate variance and bias for this fold
        variance_fold = np.var(y_pred)
        bias2_fold = np.mean((np.mean(y_pred) - y_test) ** 2) #𝐵𝑖𝑎𝑠2 = 𝐸[( 𝐸[𝑔(𝑥)] − 𝑓(𝑥) )^2 ]
        total_error_fold = variance_fold + bias2_fold #𝐸𝑟𝑟𝑜𝑟 𝑜𝑓 𝑡ℎ𝑒 𝑚𝑜𝑑𝑒𝑙 = 𝐵𝑖𝑎𝑠2 + 𝑉𝑎𝑟𝑖𝑎𝑛𝑐𝑒 + 𝐼𝑟𝑟𝑒𝑑𝑢𝑐𝑖𝑏𝑙𝑒 𝐸𝑟𝑟𝑜𝑟

        # Calculate R2 and MSE for this fold
        r2_fold = r2_score(y_test, y_pred)
        mse_fold = mean_squared_error(y_test, y_pred)

        # Append results to lists
        variance.append(variance_fold)
        bias2.append(bias2_fold)
        total_error.append(total_error_fold)
        models.append(model)

        # Print results for this fold
        print("Variance: {:.4f}, Bias2: {:.4f}, Total error: {:.4f}, R^2: {:.4f}, MSE: {:.4f}".format(variance_fold, bias2_fold, total_error_fold, r2_fold, mse_fold))

    # print the total_error of the best model
    min_error_index = np.argmin(total_error)
    best_model = models[min_error_index]
    print("Total error of the best model: {:.4f}".format(total_error[min_error_index]))

    # Testing the final model on the test data
    X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=test_size, random_state=42)
    y_pred = best_model.predict(X_test)

    # Store mse and r2 score of the model applied on the test data
    test_mse = mean_squared_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)

    print("Test MSE: {:.4f}".format(test_mse))
    print("Test R^2: {:.4f}".format(test_r2))

    return test_mse, best_model, total_error[min_error_index] # Can be modified to return whatever we need

In [ ]:
# Example usage:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing()
# print the attributes of the dataset
# X = california_housing.data[:, [1, 3, 4]]
X = california_housing.data[:, [0, 2, 3]]
y = california_housing.target

# print the 0th attribute of the dataset
print(X[:, 0])
print(california_housing.data[:, 0])


mse_list = []
best_model_list = []
te_list = []

degrees = range(1, 6)  # Try degrees from 1 to 5
for degree in degrees:
    mse , best_model , te = polynomial_regression(degree=degree, X=X, y=y, folds = 5 ,test_size=0.25, random_state=42 )
    print("Degree:", degree, "MSE:", mse)
    print("\n")
    mse_list.append(mse)
    best_model_list.append(best_model)
    te_list.append(te)

[8.3252 8.3014 7.2574 ... 1.7    1.8672 2.3886]
[8.3252 8.3014 7.2574 ... 1.7    1.8672 2.3886]
Variance: 0.6159, Bias2: 1.1592, Total error: 1.7751, R^2: 0.4923, MSE: 0.5454
Variance: 0.6857, Bias2: 1.2147, Total error: 1.9004, R^2: 0.4238, MSE: 0.6744
Variance: 0.8001, Bias2: 1.4449, Total error: 2.2450, R^2: 0.4928, MSE: 0.7296
Variance: 0.4341, Bias2: 1.1848, Total error: 1.6190, R^2: 0.3705, MSE: 0.7385
Variance: 0.7587, Bias2: 1.4722, Total error: 2.2309, R^2: 0.5437, MSE: 0.6649
Total error of the best model: 1.6190
Test MSE: 0.6592
Test R^2: 0.5018
Degree: 1 MSE: 0.6592342970938996


Variance: 1.4212, Bias2: 1.1390, Total error: 2.5602, R^2: 0.0108, MSE: 1.0627
Variance: 0.6013, Bias2: 1.1999, Total error: 1.8012, R^2: 0.4689, MSE: 0.6217
Variance: 0.8027, Bias2: 1.4435, Total error: 2.2462, R^2: 0.5090, MSE: 0.7064
Variance: 0.5257, Bias2: 1.1828, Total error: 1.7085, R^2: 0.3850, MSE: 0.7215
Variance: 0.8059, Bias2: 1.4748, Total error: 2.2808, R^2: 0.5672, MSE: 0.6307
Total 